In [1]:
import numpy as np
import scipy.stats as sps
from tqdm import tqdm

# Проверка гипотезы однородности для нормальных выборок

In [2]:
def ftest(x, y):
    """ Критерий Фишера """
    
    fstat = x.var(ddof=1) / y.var(ddof=1)
    pvalue = 2 * np.min([sps.f(len(x)-1, len(y)-1).sf(fstat), 
                         sps.f(len(x)-1, len(y)-1).cdf(fstat)])
    return pvalue

Параметры для всех запусков

In [ ]:
size = 100  # размер выборки
n_iter = 200000  # количество итераций в одном эксперименте
alpha = 0.05  # уровень значимости

### Эксперимент 1

Проверяем, что критерий Стьюдента правильно работает, т.е. верная гипотеза отвергается не более чем в $\alpha$ случаях.

In [3]:
n_errors = 0

for _ in tqdm(range(n_iter)):
    x, y = sps.norm.rvs(size=(2, size))
    n_errors += sps.ttest_ind(x, y, equal_var=True)[1] < alpha
    
print('{:.4f} +/- {:.4f}'.format(n_errors / n_iter,
                                 2 * np.sqrt(alpha / n_iter)))

100%|██████████| 200000/200000 [00:45<00:00, 4348.58it/s]


0.0503 +/- 0.0010


### Эксперимент 2

Проверяем, что критерий Фишера правильно работает

In [4]:
n_errors = 0

for _ in tqdm(range(n_iter)):
    x, y = sps.norm.rvs(size=(2, size))
    n_errors += ftest(x, y) < alpha
    
print('{:.4f} +/- {:.4f}'.format(n_errors / n_iter,
                                 2 * np.sqrt(alpha / n_iter)))

100%|██████████| 200000/200000 [05:16<00:00, 631.78it/s]


0.0505 +/- 0.0010


### Эксперимент 3

Для пары выборок применим сначала критерий Фишера.
Если он не отвергает гипотезу, то применим критерий Стьюдента.
Поправку на множественную проверку делать не будем.
Как видим, реальный уровень значимости полученного критерия почти в 2 раза превышает заявленный.

In [5]:
n_errors = 0

for _ in tqdm(range(n_iter)):
    x, y = sps.norm.rvs(size=(2, size))
    is_reject = ftest(x, y) < alpha
    n_errors += is_reject or (sps.ttest_ind(x, y, equal_var=True)[1] < alpha)
    
print('{:.4f} +/- {:.4f}'.format(n_errors / n_iter,
                                 2 * np.sqrt(alpha / n_iter)))

100%|██████████| 200000/200000 [06:50<00:00, 487.68it/s]


0.0978 +/- 0.0010


### Эксперимент 4

Теперь применим множественную проверку гипотез.
Для этого достаточно сравнить минимальное pvalue со значением $\alpha / 2$.
Тут реальный уровень значимости соответствует заявленному.

In [6]:
n_errors = 0

for _ in tqdm(range(n_iter)):
    x, y = sps.norm.rvs(size=(2, size))
    pval1 = ftest(x, y) 
    pval2 = sps.ttest_ind(x, y, equal_var=True)[1]
    n_errors += min(pval1, pval2) < alpha / 2
    
print('{:.4f} +/- {:.4f}'.format(n_errors / n_iter,
                                 2 * np.sqrt(alpha / n_iter)))

100%|██████████| 200000/200000 [06:55<00:00, 481.76it/s]


0.0494 +/- 0.0010


---------

Прикладная статистика и анализ данных, 2019

Никита Волков

https://mipt-stats.gitlab.io/